In [3]:
import requests

In [ ]:
import os
from collections import Counter

# default
import pandas as pd
import numpy as np

# 前処理
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

# モデル
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

# チューニング
# optuna

# 評価
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score


In [ ]:
# 定数
DIR_PATH = "drive/MyDrive/Colab Notebooks/datasets/livedoor-news"
FILE_PATH = os.path.join(DIR_PATH, "data.csv")
CONTENT_MAX_LEN = 128
RANDOM_STATE = 42

In [ ]:
df = pd.read_csv(FILE_PATH)
df = df[["content", "content_for_ml", "content_for_nn", "category"]]
df.head()

,content,content_for_ml,content_for_nn,category
0,世界各国で公開され、全米を始め各国で記録ラッシュが続いている映画『アベンジャーズ』の第2弾と...,世界 各国 で 公開 さ れ 、 全米 を 始め 各国 で 記録 ラッシュ が 続い て い...,世界 各国 で 公開 さ れ 、 全米 を 始め 各国 で 記録 ラッシュ が 続い て い...,movie-enter
1,日本中に大ブームを巻き起こした名作「ひみつのアッコちゃん」の実写映画化。アッコちゃん役の綾瀬...,日本 中 に 大 ブーム を 巻き起こし た 名作 「 ひみ つ の アッコ ちゃん 」 の...,日本 中 に 大 ブーム を 巻き起こし た 名作 「 ひみ つ の アッ ##コ ちゃん ...,movie-enter
2,全米では有料ケーブルテレビでのオンエアにも関わらずドラマ歴代最高視聴率を記録。すでに一部ファ...,全米 で は 有料 ケーブル テレビ で の オンエア に も 関わら ず ドラマ 歴代 最...,全米 で は 有料 ケーブル テレビ で の オンエア に も 関わら ず ドラマ 歴代 最...,movie-enter
3,今年9月に全米3大ネットワークの1つ「CBS」で放送開始と共に、瞬く間に「THEEVENT/...,今年 9 月 に 全米 3 大 ネットワーク の 1 つ 「 CBS 」 で 放送 開始 と...,今年 9 月 に 全米 3 大 ネットワーク の 1 つ 「 CBS 」 で 放送 開始 と...,movie-enter
4,ペ・ドゥナとソ・ジソクの主演で贈る、遅咲きの歌姫が恋と夢のステージで輝く人生逆転シンデレラ・...,ペ・ドゥナ と ソ・ジソク の 主演 で 贈る 、 遅咲き の 歌姫 が 恋 と 夢 の ス...,ペ ##・ ##ドゥ ##ナ と ソ ##・ ##ジ ##ソ ##ク の 主演 で 贈る 、...,movie-enter


In [ ]:
df.query("content.str.contains('明治大学を卒業した学生は')", engine="python")

,content,content_for_ml,content_for_nn,category
2115,週刊現代2012年1月21日号の記事「明治大学を卒業した学生は謙虚で、会社で人気」がネット掲...,週刊 現代 2012 年 1 月 21 日 号 の 記事 「 明治大学 を 卒業 し た 学...,週刊 現代 2012 年 1 月 21 日 号 の 記事 「 明治大学 を 卒業 し た 学...,topic-news


In [ ]:
tmp = df.sample(1)
tmp.content.values[0]

'1985年に公開された映画『バック・トゥ・ザ・フューチャー』でタイムマシンとして登場し、デロリアンとして知られる名車、DMC-12を電気自動車にした“DMCEV”の仮予約受付が開始された。デジタル家電商品企画のレッドスターとアメリカのDeLoreanMotorCompanyが正規ディストリビューター契約を締結したことで、映画ファンにはたまらない電気自動車が実現する。初回生産台数は、世界で30台限定。限定台数に達し次第、締め切られる。今回の仮予約受付は、日本市場のマーケティング活動を主な目的としており、予約金等は一切不要。まだアメリカでは受付を開始しておらず、日本での受付が世界最速となる。レッドスター・グループ代表の松川政裕氏は、「僕がふと、デロリアンの電気自動車版と市販を思いついたのは、2010年の6月だった。早速、米国のDMC本社と日本のDMC-Japanにアイデアを送った。返事はきたが、電気自動車という突拍子もないアイデアに戸惑っている様子だった。でも、粘り強い説得と世間の流れに皆やる気をだしてくれた。自分の思いつきのアイデアで、デロリアンEVが生まれ、世界中で販売されることに大変興奮しています」とコメントしている。『バック・トゥ・ザ・フューチャー2』に登場するスニーカー“NIKEMAG”が販売されるなど、夢でしかないと思われていたものが実現化されてきている。同作に登場する、宙に浮くスケートボード“ホバーボード”が商品化される日も近いかもしれない。デロリアンEVは、12月3日より、オフィシャルホームページで仮予約をすることが出来る。価格は未定。2013年までに納車される。・“デロリアンEV”仮予約オフィシャルページ'

In [ ]:
tmp.category.values[0]

'movie-enter'

# 前処理

In [ ]:
df["content_for_ml"] = df.content_for_ml.str.split(" ").str[:CONTENT_MAX_LEN].str.join(" ")
df["content_for_nn"] = df.content_for_nn.str.split(" ").str[:CONTENT_MAX_LEN].str.join(" ")

In [1]:
df.iloc[df.content_for_ml.str.split(" ").str.len().argmin()]

NameError: ignored

In [ ]:
# DFを計算
counter = Counter(" ".join(df.content_for_ml.str.split(" ").map(set).str.join(" ").values).split(" "))
counter_df = pd.DataFrame(counter.items(), columns=["word", "cnt"])

In [ ]:
counter_df["df"] = counter_df.cnt / len(df)
counter_df.head()

,word,cnt,df
0,から,3290,0.446586
1,こと,3094,0.419981
2,し,6318,0.857608
3,超,214,0.029048
4,れる,1504,0.204154


In [ ]:
counter_df.shape, counter_df.query("cnt == 1").shape, counter_df.query("cnt == 2").shape

((36296, 3), (16628, 3), (5121, 3))

In [ ]:
counter_df.describe()

,cnt,df
count,36296.000000,36296.000000
mean,16.089845,0.002184
std,161.951288,0.021983
min,1.000000,0.000136
25%,1.000000,0.000136
50%,2.000000,0.000271
75%,5.000000,0.000679
max,7336.000000,0.995792


In [ ]:
cat_le = LabelEncoder()
df["category_id"] = cat_le.fit_transform(df.category)
print({k:v for k, v in enumerate(cat_le.classes_)})
df.head()

{0: 'dokujo-tsushin', 1: 'it-life-hack', 2: 'kaden-channel', 3: 'livedoor-homme', 4: 'movie-enter', 5: 'peachy', 6: 'smax', 7: 'sports-watch', 8: 'topic-news'}


,content,content_for_ml,content_for_nn,category,category_id
0,世界各国で公開され、全米を始め各国で記録ラッシュが続いている映画『アベンジャーズ』の第2弾と...,世界 各国 で 公開 さ れ 、 全米 を 始め 各国 で 記録 ラッシュ が 続い て い...,世界 各国 で 公開 さ れ 、 全米 を 始め 各国 で 記録 ラッシュ が 続い て い...,movie-enter,4
1,日本中に大ブームを巻き起こした名作「ひみつのアッコちゃん」の実写映画化。アッコちゃん役の綾瀬...,日本 中 に 大 ブーム を 巻き起こし た 名作 「 ひみ つ の アッコ ちゃん 」 の...,日本 中 に 大 ブーム を 巻き起こし た 名作 「 ひみ つ の アッ ##コ ちゃん ...,movie-enter,4
2,全米では有料ケーブルテレビでのオンエアにも関わらずドラマ歴代最高視聴率を記録。すでに一部ファ...,全米 で は 有料 ケーブル テレビ で の オンエア に も 関わら ず ドラマ 歴代 最...,全米 で は 有料 ケーブル テレビ で の オンエア に も 関わら ず ドラマ 歴代 最...,movie-enter,4
3,今年9月に全米3大ネットワークの1つ「CBS」で放送開始と共に、瞬く間に「THEEVENT/...,今年 9 月 に 全米 3 大 ネットワーク の 1 つ 「 CBS 」 で 放送 開始 と...,今年 9 月 に 全米 3 大 ネットワーク の 1 つ 「 CBS 」 で 放送 開始 と...,movie-enter,4
4,ペ・ドゥナとソ・ジソクの主演で贈る、遅咲きの歌姫が恋と夢のステージで輝く人生逆転シンデレラ・...,ペ・ドゥナ と ソ・ジソク の 主演 で 贈る 、 遅咲き の 歌姫 が 恋 と 夢 の ス...,ペ ##・ ##ドゥ ##ナ と ソ ##・ ##ジ ##ソ ##ク の 主演 で 贈る 、...,movie-enter,4


In [ ]:
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(df.content_for_ml, df.category_id, test_size=0.2, random_state=RANDOM_STATE, stratify=df.category_id)
X_train_nn, X_test_nn, y_train_nn, y_test_nn = train_test_split(df.content_for_nn, df.category_id, test_size=0.2, random_state=RANDOM_STATE, stratify=df.category_id)
X_train_ml.shape, X_train_nn.shape, X_test_ml.shape, X_test_nn.shape

((5893,), (5893,), (1474,), (1474,))

In [ ]:
X_test_ml.head()

7361    vol . 3 : 谷中 ・ 根津 ・ 千駄木 の 「 谷根 千 」 下町 地域 Camer...
7258    転職 者 なら 誰 でも 気 に なる 採用 する 側 の 心理 。 しかし 、 実際 に ...
5614    「 もともと 子供 嫌い で 、 友達 とか が 子供 を 連れ て き て も 、 どう ...
4198    今年 は 、 女子 サッカー ワールドカップ で の 「 なでしこ ジャパン 」 優勝 や ...
2174    7 日 、 NEWS ポスト セブン が 配信 し た 「 ブラック 企業 OL 内定 式 ...
Name: content_for_ml, dtype: object

In [ ]:
X_train_ml.head()

6657    最近 の パソコン の 多く は 中国 など の 外国 で 生産 さ れ て いる 。 コス...
2933    地上 の 基地 局 で は なく 、 地球 の 周り を 周回 移動 する 衛星 を 介し ...
1542    ARROWSMeF - 11 D が 8 月 15 日 に 発売 予定 ! NTT ドコモ ...
3513    今年 話題 の 恋愛 ・ スキャンダル ニュース 今年 2 月 に モデル 兼 タレント の...
5740    昨年 、 新聞 や テレビ で 話題 に なっ た 、 厚生 労働省 による 「 相対 的 ...
Name: content_for_ml, dtype: object

In [ ]:
X_train_nn.head()

6657    最近 の パソコン の 多く は 中国 など の 外国 で 生産 さ れ て いる 。 コス...
2933    地上 の 基地 局 で は なく 、 地球 の 周り を 周回 移動 する 衛星 を 介し ...
1542    AR ##RO ##WS ##M ##e ##F - 11 D が 8 月 15 日 に 発...
3513    今年 話題 の 恋愛 ・ スキャンダル ニュース 今年 2 月 に モデル 兼 タレント の...
5740    昨年 、 新聞 や テレビ で 話題 に なっ た 、 厚生 労働省 による 「 相対 的 ...
Name: content_for_nn, dtype: object

# 学習


In [ ]:
def create_simple_pipeline(clf):
  return Pipeline(
      [# dfが1の単語を予測に使うのは精度が上がることがあっても良くない気が
       ("vect", CountVectorizer(min_df=2, max_df=0.5)),
       ("clf", clf)
      ]
  )

def create_gscv_pipeline(clf, clf_param_grid : dict, is_refit=True):
  return GridSearchCV(
      Pipeline(
          [
           ("vect", TfidfVectorizer()),
           ("clf", clf)
          ]
      ),
      [# パラメータ設定
          { # TfidfVectorizer
           'vect__ngram_range': [(1, 1)], # (1,2), (2,2)も追加しても良いかも
           'vect__max_df': [0.01, 0.1, 0.5], # 結構大事,
           'vect__min_df': [2, 5, 10],
           **clf_param_grid
          },
          # CountVectorizerのように使える
          {
              'vect__ngram_range': [(1, 1)],
              'vect__max_df': [0.01, 0.1, 0.5],
              'vect__min_df': [2, 5, 10], 
              'vect__use_idf':[False],
              'vect__norm':[None],
              **clf_param_grid
          }
      ],
      scoring="f1_macro",
      cv=5,
      verbose=2,
      n_jobs=-1,
      refit=is_refit
  )



In [ ]:
def evaluate_model(model):
  model.fit(X_train_ml, y_train_ml)
  train_pred = model.predict(X_train_ml)
  test_pred = model.predict(X_test_ml)
  print("train_acc", accuracy_score(y_train_ml, train_pred))
  print("train_f1_macro", f1_score(y_train_ml, train_pred, average="macro")) # microはaccと同じ
  print("test_acc", accuracy_score(y_test_ml, test_pred))
  print("test_f1_macro", f1_score(y_test_ml, test_pred, average="macro"))


### ナイーブベイズ

In [ ]:
# mecab
nb_model = create_simple_pipeline(MultinomialNB())
evaluate_model(nb_model)

train_acc 0.9348379433225861
train_f1_macro 0.9332783015666691
test_acc 0.8168249660786974
test_f1_macro 0.805915333174073


In [ ]:
# # sentence piece
# nb_model = create_simple_pipeline(MultinomialNB())
# nb_model.fit(X_train_nn, y_train_nn)
# nb_model.score(X_test_nn, y_test_nn)

In [ ]:
nb_tune_model = create_gscv_pipeline(
    MultinomialNB(), {"clf__alpha": [0.01, 0.1, 1]})
evaluate_model(nb_tune_model)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   17.4s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  1.9min finished


train_acc 0.9348379433225861
train_f1_macro 0.9332783015666691
test_acc 0.8168249660786974
test_f1_macro 0.805915333174073


In [ ]:
nb_tune_model.best_params_

{'clf__alpha': 1,
 'vect__max_df': 0.5,
 'vect__min_df': 2,
 'vect__ngram_range': (1, 1),
 'vect__norm': None,
 'vect__use_idf': False}

# ロジスティック回帰

In [ ]:
GridSearchCV?

In [ ]:
lr_model = create_simple_pipeline(
    LogisticRegression(random_state=RANDOM_STATE, max_iter=200))
evaluate_model(lr_model)

train_acc 0.9998303071440693
train_f1_macro 0.9998397791054049
test_acc 0.850067842605156
test_f1_macro 0.8412139520333397


In [ ]:
# lr_model = create_simple_pipeline(LogisticRegression(random_state=RANDOM_STATE, max_iter=200))
# lr_model.fit(X_train_nn, y_train_nn)
# lr_model.score(X_test_nn, y_test_nn)

In [ ]:
re_lr_tune_model = create_gscv_pipeline(
    LogisticRegression(random_state=RANDOM_STATE, max_iter=200),
    {"clf__C": [0.1, 1, 10]},
    is_refit=False
    )

In [ ]:
evaluate_model(re_lr_tune_model)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  7.9min finished


NotFittedError: ignored

In [ ]:
lr_tune_model = create_gscv_pipeline(
    LogisticRegression(random_state=RANDOM_STATE, max_iter=200),
    {"clf__C": [0.1, 1, 10]})
evaluate_model(lr_tune_model)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  7.4min finished


train_acc 0.9994909214322077
train_f1_macro 0.9995166051624208
test_acc 0.8616010854816825
test_f1_macro 0.851625114350143


In [ ]:
tmp_pred = lr_tune_model.best_estimator_.predict(X_train_ml)
tmp_acc = tmp_pred == y_train_ml
print(tmp_acc.sum() / tmp_acc.shape[0])

0.9994909214322077


In [ ]:
lr_tune_model.best_params_, lr_tune_model.best_score_

({'clf__C': 10,
  'vect__max_df': 0.5,
  'vect__min_df': 2,
  'vect__ngram_range': (1, 1)},
 0.8489160973688532)

## RF

In [ ]:
rfc_model = create_simple_pipeline(RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=200))
evaluate_model(rfc_model)

train_acc 0.9998303071440693
train_f1_macro 0.9998397807711166
test_acc 0.8039348710990502
test_f1_macro 0.7828904597177851


In [ ]:
rfc_tune_model = create_gscv_pipeline(
    RandomForestClassifier(random_state=RANDOM_STATE),
    {"clf__n_estimators": [200]})
evaluate_model(rfc_tune_model)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 10.1min finished


train_acc 0.9998303071440693
train_f1_macro 0.9998397807711166
test_acc 0.7951153324287653
test_f1_macro 0.7766687885692655


## 勾配ブースティング

In [ ]:
# lgbc_model = create_simple_pipeline(lgb.LGBMClassifier(random_state=RANDOM_STATE))
# evaluate_model(lgbc_model)

In [ ]:
lgbc_tune_model = create_gscv_pipeline(
    lgb.LGBMClassifier(random_state=RANDOM_STATE),
    {})
evaluate_model(lgbc_tune_model)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 14.5min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 28.1min finished


train_acc 0.9996606142881385
train_f1_macro 0.999680136621994
test_acc 0.8310719131614654
test_f1_macro 0.8213408017170123


In [ ]:
import pickle
BIN_PATH = os.path.join(DIR_PATH, "models.bin")
with open(BIN_PATH, "wb") as f:
  pickle.dump([nb_tune_model, lr_tune_model, rfc_tune_model, lgbc_tune_model], f)